In [351]:
import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import seaborn as sns 
sns.set()

This research proposal will investigate causes of death in US medical facilities and how they have changed from 2008 vs 2018. This study will also include race, age and death statistics by US census regions. With major advances in many areas of healthcare over the last decade, the purpose of the study is evaluate whether this data can improve decisions related to region-specific future hospital center expansions. 

Query Criteria:\
Title:	US Cause of death 2008 and 2018 in medical facilities in two year increments

Place of Death: Medical Facility - Inpatient
                
Year/Month:	2008, 2010, 2012, 2014, 2016, and 2018\
Group by: Cause of death; Ten-Year Age Groups; Gender\
Calculate Rates Per:	100,000\
Basic search criteria example:\
https://wonder.cdc.gov/controller/saved/D76/D82F820

Also a good resource to understand death rate calculations:
https://www.stats.indiana.edu/vitals/CalculatingARate.pdf


Centers for Disease Control and Prevention, National Center for Health Statistics. Underlying Cause of Death 1999-2018 on CDC WONDER Online Database, released in 2020. Data are from the Multiple Cause of Death Files, 1999-2018, as compiled from data provided by the 57 vital statistics jurisdictions through the Vital Statistics Cooperative Program. Accessed at http://wonder.cdc.gov/ucd-icd10.html on May 1, 2020 5:00:25 PM


In [352]:
# Local downloads from mulitple queries were required due to limits and were moved to remote repository

dfcod = pd.read_csv('https://raw.githubusercontent.com/robholmstrom/Sams_github/master/Capstone_1/COD_csv_files/US_2008-2018_COD_chapters.csv')
dfcod.info()
dfcod.head(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6790 entries, 0 to 6789
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Notes                       80 non-null     object 
 1   ICD-10 113 Cause List       6710 non-null   object 
 2   ICD-10 113 Cause List Code  6710 non-null   object 
 3   Ten-Year Age Groups         6710 non-null   object 
 4   Ten-Year Age Groups Code    6710 non-null   object 
 5   Gender                      6710 non-null   object 
 6   Gender Code                 6710 non-null   object 
 7   Year                        6710 non-null   float64
 8   Year Code                   6710 non-null   float64
 9   Deaths                      6710 non-null   float64
 10  Population                  6710 non-null   object 
 11  Crude Rate                  6710 non-null   object 
dtypes: float64(3), object(9)
memory usage: 636.7+ KB


Notes             ICD-10 113 Cause List ICD-10 113 Cause List Code  \
0   NaN  #Salmonella infections (A01-A02)                  GR113-001   
1   NaN  #Salmonella infections (A01-A02)                  GR113-001   
2   NaN  #Salmonella infections (A01-A02)                  GR113-001   
3   NaN  #Salmonella infections (A01-A02)                  GR113-001   
4   NaN  #Salmonella infections (A01-A02)                  GR113-001   
5   NaN  #Salmonella infections (A01-A02)                  GR113-001   
6   NaN  #Salmonella infections (A01-A02)                  GR113-001   
7   NaN  #Salmonella infections (A01-A02)                  GR113-001   
8   NaN  #Salmonella infections (A01-A02)                  GR113-001   
9   NaN  #Salmonella infections (A01-A02)                  GR113-001   

  Ten-Year Age Groups Ten-Year Age Groups Code  Gender Gender Code    Year  \
0            < 1 year                        1  Female           F  2010.0   
1            < 1 year                        1  Female           F  2012.0   
2            < 1 year                        1    Male           M  2008.0   
3            < 1 year                        1    Male           M  2012.0   
4            < 1 year                        1    Male           M  2014.0   
5            < 1 year                        1    Male           M  2018.0   
6           1-4 years                    4-Jan  Female           F  2008.0   
7           1-4 years                    4-Jan  Female           F  2012.0   
8           1-4 years                    4-Jan  Female           F  2014.0   
9           1-4 years                    4-Jan    Male           M  2012.0   

   Year Code  Deaths      Population      Crude Rate  
0     2010.0     1.0  Not Applicable  Not Applicable  
1     2012.0     1.0  Not Applicable  Not Applicable  
2     2008.0     2.0  Not Applicable  Not Applicable  
3     2012.0     1.0  Not Applicable  Not Applicable  
4     2014.0     2.0  Not Applicable  Not Applicable  
5     2018.0     2.0  Not Applicable  Not Applicable  
6     2008.0     1.0  Not Applicable  Not Applicable  
7     2012.0     1.0  Not Applicable  Not Applicable  
8     2014.0     1.0  Not Applicable  Not Applicable  
9     2012.0     1.0  Not Applicable  Not Applicable

In [353]:


# Get rid of unnecessary columns that were adding null values or just noise

dfcod.drop(['Gender Code','Notes', 'Population', 'Ten-Year Age Groups Code', 'Crude Rate', 'Year Code', 'ICD-10 113 Cause List Code'], axis = 1, inplace = True)
dfcod.dropna(inplace = True)

# Rename some columns for clarity:

dfcod.rename(columns = {'Deaths': 'Deaths per 100,000', 
                        'Ten-Year Age Groups':'Age range', 
                        'ICD-10 113 Cause List': 'Cause of death'}, 
                          inplace = True)

# Add column for ICD chapter
dfcod.
dfcod.info() # Ensure that all columns do not contain null values



<class 'pandas.core.frame.DataFrame'>
Int64Index: 6710 entries, 0 to 6709
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Cause of death      6710 non-null   object 
 1   Age range           6710 non-null   object 
 2   Gender              6710 non-null   object 
 3   Year                6710 non-null   float64
 4   Deaths per 100,000  6710 non-null   float64
dtypes: float64(2), object(3)
memory usage: 314.5+ KB


In [354]:
dfcod.head(10) # Example of table without the total population data--more clean up will happen later on
# .map method to group categories



Cause of death  Age range  Gender    Year  \
0  #Salmonella infections (A01-A02)   < 1 year  Female  2010.0   
1  #Salmonella infections (A01-A02)   < 1 year  Female  2012.0   
2  #Salmonella infections (A01-A02)   < 1 year    Male  2008.0   
3  #Salmonella infections (A01-A02)   < 1 year    Male  2012.0   
4  #Salmonella infections (A01-A02)   < 1 year    Male  2014.0   
5  #Salmonella infections (A01-A02)   < 1 year    Male  2018.0   
6  #Salmonella infections (A01-A02)  1-4 years  Female  2008.0   
7  #Salmonella infections (A01-A02)  1-4 years  Female  2012.0   
8  #Salmonella infections (A01-A02)  1-4 years  Female  2014.0   
9  #Salmonella infections (A01-A02)  1-4 years    Male  2012.0   

   Deaths per 100,000  
0                 1.0  
1                 1.0  
2                 2.0  
3                 1.0  
4                 2.0  
5                 2.0  
6                 1.0  
7                 1.0  
8                 1.0  
9                 1.0

In [389]:
groupedcod = dfcod.groupby('Cause of death').mean()[['Deaths per 100,000']].reset_index()
plt.figure(figsize=(5,30))
plt.xticks(rotation=90)
plt.barh(groupedcod['Cause of death'], width=groupedcod['Deaths per 100,000'])

<BarContainer object of 71 artists>

Below, I want to conduct exploratory analysis of the dfcod. Want to look summarized unique list of causes of death in order to condense

In [411]:
# Use this to convert cause of death column to a list: list = df['Name'].values.tolist()
cod = dfcod['Cause of death'].values.tolist()

# Get unique cods and place into a dataframe column. Will populate adjacent column with broader designation

codunique = [] 
for x in cod: # Generate unique list with all causes of deaths
    if x not in codunique:
        codunique.append(x)

icd10 = []
for x in codunique: # Generate list with all ICD codes in new column 
    icd10.append(x.split(' ')[len(x.split(' '))-1])

icdchapter =
coddf = pd.DataFrame({'Unique causes of death':codunique, 'ICD10 code':icd10})

#Temporarily set display of rows to max in order to view all ICD coded causes of death
pd.set_option('display.max_rows', None)
coddf



Unique causes of death  \
0                    #Salmonella infections (A01-A02)   
1                #Shigellosis and amebiasis (A03,A06)   
2                             #Tuberculosis (A16-A19)   
3                               #Whooping cough (A37)   
4             #Scarlet fever and erysipelas (A38,A46)   
5                      #Meningococcal infection (A39)   
6                               #Septicemia (A40-A41)   
7                                 #Syphilis (A50-A53)   
8   #Arthropod-borne viral encephalitis (A83-A84,A...   
9                                      #Measles (B05)   
10                         #Viral hepatitis (B15-B19)   
11  #Human immunodeficiency virus (HIV) disease (B...   
12                                 #Malaria (B50-B54)   
13                     #Malignant neoplasms (C00-C97)   
14  Malignant neoplasms of lip, oral cavity and ph...   
15              Malignant neoplasm of esophagus (C15)   
16                Malignant neoplasm of stomach (C16)   
17  Malignant neoplasms of colon, rectum and anus ...   
18  Malignant neoplasms of liver and intrahepatic ...   
19               Malignant neoplasm of pancreas (C25)   
20                 Malignant neoplasm of larynx (C32)   
21  Malignant neoplasms of trachea, bronchus and l...   
22                   Malignant melanoma of skin (C43)   
23                 Malignant neoplasm of breast (C50)   
24           Malignant neoplasm of cervix uteri (C53)   
25  Malignant neoplasms of corpus uteri and uterus...   
26                  Malignant neoplasm of ovary (C56)   
27               Malignant neoplasm of prostate (C61)   
28  Malignant neoplasms of kidney and renal pelvis...   
29                Malignant neoplasm of bladder (C67)   
30  Malignant neoplasms of meninges, brain and oth...   
31  Malignant neoplasms of lymphoid, hematopoietic...   
32                              Hodgkin disease (C81)   
33                     Non-Hodgkin lymphoma (C82-C85)   
34                                 Leukemia (C91-C95)   
35  Multiple myeloma and immunoproliferative neopl...   
36  #In situ neoplasms, benign neoplasms and neopl...   
37                                 #Anemias (D50-D64)   
38                       #Diabetes mellitus (E10-E14)   
39                #Nutritional deficiencies (E40-E64)   
40                              #Meningitis (G00,G03)   
41                       #Parkinson disease (G20-G21)   
42                           #Alzheimer disease (G30)   
43       #Diseases of heart (I00-I09,I11,I13,I20-I51)   
44  #Essential hypertension and hypertensive renal...   
45                #Cerebrovascular diseases (I60-I69)   
46                             #Atherosclerosis (I70)   
47              #Aortic aneurysm and dissection (I71)   
48                 #Influenza and pneumonia (J09-J18)   
49      #Chronic lower respiratory diseases (J40-J47)   
50  #Pneumoconioses and chemical effects (J60-J66,...   
51       #Pneumonitis due to solids and liquids (J69)   
52                            #Peptic ulcer (K25-K28)   
53                    #Diseases of appendix (K35-K38)   
54                                  #Hernia (K40-K46)   
55  #Chronic liver disease and cirrhosis (K70,K73-...   
56  #Cholelithiasis and other disorders of gallbla...   
57  #Nephritis, nephrotic syndrome and nephrosis (...   
58        #Infections of kidney (N10-N12,N13.6,N15.1)   
59                     #Hyperplasia of prostate (N40)   
60  #Inflammatory diseases of female pelvic organs...   
61  #Pregnancy, childbirth and the puerperium (O00...   
62  #Certain conditions originating in the perinat...   
63  #Congenital malformations, deformations and ch...   
64  #Accidents (unintentional injuries) (V01-X59,Y...   
65  #Intentional self-harm (suicide) (*U03,X60-X84...   
66      #Assault (homicide) (*U01-*U02,X85-Y09,Y87.1)   
67                    #Legal intervention (Y35,Y89.0)   
68  #Operations of war and their sequelae (Y36,Y89.1)   
69  #Complications of medical and surgica

In [356]:
# Reset display options to 10
pd.set_option('display.max_rows', 10)